In [32]:
# coding: utf-8
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_curve
import operator
from matplotlib import pylab as plt
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from collections import defaultdict

# ---read data---
path = '../data/'

raw_feat = pd.read_csv(path + 'newfeat/merge_data.csv',sep=',', header = 'infer',encoding='UTF-8')


# ---area feature---
area = ['东北','华北','华中','华东','西北','西南','华南','境外']

loc  = defaultdict(list)
loc[0] = ['东北','辽宁','吉林','黑龙江','长白山','北戴河','松花湖','老虎滩','锡林郭勒草原','沈阳故宫',
          '沈阳','大连','鞍山','抚顺','本溪','丹东','锦州','营口','阜新','辽阳','盘锦','铁岭','朝阳','葫芦岛','兴城','海城',
          '长春','四平','松原','白城','辽源','通化','延边','梅河口','公主岭','长影世纪城','净月潭','伪满','查干湖','松花湖',
          '清昭陵','清福陵','星海广场','千山','鸭绿江','本溪水洞' ,'甘井子','大工','东财','辽师','吉大',
          '太阳岛','亚布力','中央大街','五大连池','扎龙湿地','镜泊湖', '哈工大','哈工程','黑大','哈医大',
          '哈尔滨', '齐齐哈尔','鸡西','鹤岗','双鸭山','大庆','伊春','佳木斯','七台河','牡丹江','黑河','绥化','大兴安岭']
          
loc[1] = [ '华北','北京','天津','河北','山西','内蒙古','呼和浩特','包头','鄂尔多斯','乌兰察布','冀','黄土高原',
          '故宫','颐和园','五台山','平遥古城','云冈石窟','野三坡','白洋淀','晋','北大','清华','南开','天大',
          '保定','唐山','太原','大慈阁','北戴河','清东陵','白洋淀','避暑山庄','西柏坡','山海关',
         '石家庄','辛集','藁城','晋州','新乐','鹿泉','遵化','迁安','秦皇岛','邯郸','武安','邢台','南宫','沙河','涿州','定州',
          '安国','高碑店','张家口','承德','沧州','泊头','任丘','黄骅','河间','廊坊','霸州','三河','衡水','冀州','深州']

loc[2] = ['华中','河南','湖北','湖南','中原','少林寺','龙门石窟','嵩山','黄鹤楼','神农架','张家界','衡山',
          '郑州','殷墟','天地之中','黄帝故里','白马寺',
          '开封','洛阳','南阳','漯河','许昌','三门峡','平顶山','周口','驻马店','新乡','鹤壁','焦作','濮阳',
          '安阳','商丘','信阳','济源',
         '武汉','黄石','襄樊','十堰','荆州','宜昌','荆门','鄂州','孝感','黄冈','咸宁','随州','恩施',
          '仙桃','潜江','天门','神农架','武昌','东湖','武当山','三峡','隆中','汉口',
         '张家界','岳麓山','岳阳楼','炎帝陵','凤凰古城','韶山','崀山','东江湖',
          '长沙','湘潭','株洲','郴州','衡阳','邵阳','永州','湘西','常德','娄底','益阳','怀化']
          

loc[3] = [ '华东','上海','江苏','浙江','安徽','江西','福建','山东',
         '济南','青岛','南京','苏州','无锡','杭州','宁波','厦门','福州','烟台','合肥','南昌','泉州','温州',
         '绍兴','台州','威海','南通','泰安','东营','镇江','济宁','扬州','徐州','泰州','九江','赣州','滨州',
          '湖州','嘉兴','临沂','金华','宜春','新余','景德镇','盐城','萍乡','芜湖','连云港','聊城','蚌埠','淮南','马鞍山',
          '丽水','衢州','安庆','吉安','鹰潭','漳州','淮安','德州','菏泽','宿迁','舟山','淮北','亳州','淮北','铜陵','黄山',
          '阜阳','宿州','滁州','六安','宣城','池州','莆田','龙岩','三明','南平','宁德','抚州','上饶','淄博','枣庄','潍坊',
          '日照','莱芜','中山陵','上海外滩','杭州西湖','庐山','鼓浪屿','黄山','趵突泉','外滩','豫园','南京路','人民广场',
          '静安寺','新天地','迪士尼乐园','黄浦','卢湾','徐汇','长宁','静安','普陀','闸北','虹口','杨浦','浦东','宝山','嘉定',
          '闵行','松江','青浦','金山','南汇','奉贤','崇明']

loc[4] = [ '西北','甘肃','青海','宁夏','新疆','维吾尔','内蒙古',
          '莫高窟','麦积山','崆峒山','嘉峪关','鸣沙山','月牙泉','甘南','兰州','天水','白银','平凉','庆阳','陇南','定西',
          '金昌','武威','张掖','酒泉','嘉峪关','临夏','甘南','西宁','海东','格尔木','德令哈','玉树','天山','高昌古城','吐鲁番',
          '喀纳斯湖','博斯腾湖','魔鬼城','乌鲁木齐','克拉玛依','吐鲁番','哈密','库尔勒','阿克苏','阿图什','喀什','和田','伊宁',
          '奎屯','塔城','乌苏','阿勒泰','石河子','呼和浩特','包头','通辽','赤峰','乌海','鄂尔多斯','呼伦贝尔','乌兰察布',
          '巴彦淖尔','兴安','锡林郭勒','阿拉善','满洲里','二连浩特','乌兰浩特','扎兰屯','牙克石','锡林浩特','霍林郭勒',
          '丰镇','根河','额尔古纳','阿尔山']

loc[5] = ['西南','云南','贵州','四川','西藏','九寨','黄龙','都江堰','青城山','乐山','峨眉山','阆中古城',
          '稻城亚丁','川大','电子科大','成都','重庆','昆明','南宁','贵阳','绵阳','宜宾','南充','攀枝花','大理','遵义',
          '桂林','柳州','防城港','钦州','六盘水','自贡','德阳','泸州','玉溪','丽江','重大','云大','西财']

loc[6] =['华南','广东','广西','海南','台湾','香港','澳门','广州','韶关','深圳','珠海','汕头','佛山','江门',
         '湛江','茂名','肇庆','惠州','梅州','汕尾','河源','阳江','清远','东莞','中山','潮州','揭阳','云浮',
         '增城','从化','乐昌','南雄','台山','开平','鹤山','恩平','廉江','雷州','吴川','高州','化州','信宜','高要','四会',
         '兴宁','陆丰','阳春','英德','连州','普宁','罗定','南宁','柳州','桂林','梧州','北海','防城港','钦州','贵港','玉林',
         '百色','贺州','河池','来宾','崇左','岑溪','东兴','桂平','北流','宜州','合山','凭祥','海口','三亚','五指山','琼海',
         '儋州','文昌','万宁','台北','高雄','基隆','台中','台南','新竹','嘉义板桥','宜兰','竹北','桃园','苗栗',
         '丰原','彰化','南投','嘉义','太保','云林','斗六','新营','凤山','屏东','台东','花莲','澎湖','马公']

loc[7] = ['境外','美国','英国','法国','俄罗斯','加拿大','巴西','澳大利亚','印尼','泰国','马来西亚','新加坡',
          '菲律宾','越南','印度','日本']
featAreaPd = pd.DataFrame(columns=range(len(area)))
featAreaPd.columns = 'area_' + featAreaPd.columns.astype(str)

print (featAreaPd)
alen = len(area)


for content in raw_feat['contents']:
    tmpDict  = defaultdict(int)
    for word in content.split():
        tmpDict[word] = tmpDict[word] + 1
    tmpL  = [0] * alen
    for i in range(len(area)):
        for l in loc[i]:
            tmpL[i] = tmpL[i] + tmpDict[l]
    featAreaPd = featAreaPd.append(pd.DataFrame(np.array(tmpL).reshape(1,alen), columns = list(featAreaPd.columns)), ignore_index=True)

    
feat_area = featAreaPd
feat_area.to_csv(path+'newfeat/feat_area.csv', header='infer', sep = ',', index = 0)




Empty DataFrame
Columns: [uidarea_0, uidarea_1, uidarea_2, uidarea_3, uidarea_4, uidarea_5, uidarea_6, uidarea_7]
Index: []


In [26]:

# ---time feature---
def timePeriod(s):
    if ((s >'00:00') & (s<='03:00')):
        return 0
    elif((s >'03:00') & (s<='06:00')):
        return 1
    elif((s >'06:00') & (s<='09:00')):
        return 2
    elif((s >'09:00') & (s<='12:00')):
        return 3
    elif((s >'12:00') & (s<='15:00')):
        return 4
    elif((s >'15:00') & (s<='18:00')):
        return 5
    elif((s >'18:00') & (s<='21:00')):
        return 6
    elif((s >'21:00') & (s<='24:00')):
        return 7

train_path = path + 'train/' 
test_path = path + 'valid/'


train_status_lines = open(train_path + 'train_status.txt',encoding='UTF-8').readlines()
train_dict = dict()
early_flag = 0
for train_status in train_status_lines:
    r = train_status.split(',')
    s = r[4].split()
    
    if(len(s)==1):
        early_flag = 1    
    else:    
        t = timePeriod(s[1])
        if((r[0] in train_dict) == False):
            train_dict[r[0]] = defaultdict(int)
        train_dict[r[0]][t] = train_dict[r[0]][t] + 1
        if(early_flag==1):
            train_dict[r[0]][t] = train_dict[r[0]][t] + 1
            early_flag = 0
train_dict_pd = pd.DataFrame(train_dict).T.reset_index().rename(columns={"index": "uid"})
train_dict_pd = train_dict_pd.drop(None, axis=1)
train_dict_pd.columns = 'timePeriod_3hour_' + train_dict_pd.columns.astype(str)
train_dict_pd = train_dict_pd.rename(columns={"timePeriod_3hour_uid": "uid"})
train_dict_pd = train_dict_pd.fillna(0)

test_status_lines = open(test_path + 'valid_status.txt',encoding='UTF-8').readlines()
test_dict = dict()
early_flag = 0
for test_status in test_status_lines:
    r = test_status.split(',')
    s = r[4].split()
    
    if(len(s)==1):
        early_flag = 1    
    else:    
        t = timePeriod(s[1])
        if((r[0] in test_dict) == False):
            test_dict[r[0]] = defaultdict(int)
        test_dict[r[0]][t] = test_dict[r[0]][t] + 1
        if(early_flag==1):
            test_dict[r[0]][t] = test_dict[r[0]][t] + 1
            early_flag = 0
test_dict_pd = pd.DataFrame(test_dict).T.reset_index().rename(columns={"index": "uid"})
test_dict_pd = test_dict_pd.drop(None, axis=1)
test_dict_pd.columns = 'timePeriod_3hour_' + test_dict_pd.columns.astype(str)
test_dict_pd = test_dict_pd.rename(columns={"timePeriod_3hour_uid": "uid"})
test_dict_pd = test_dict_pd.fillna(0)

feat_time_3hour = pd.concat([train_dict_pd, test_dict_pd], axis = 0, ignore_index = True )

def timePerHour(s):
    if ((s >'00:00') & (s<='01:00')):
        return 0
    elif((s >'01:00') & (s<='02:00')):
        return 1
    elif((s >'02:00') & (s<='03:00')):
        return 2
    elif((s >'03:00') & (s<='04:00')):
        return 3
    elif((s >'04:00') & (s<='05:00')):
        return 4
    elif((s >'05:00') & (s<='06:00')):
        return 5
    elif((s >'06:00') & (s<='07:00')):
        return 6
    elif((s >'07:00') & (s<='08:00')):
        return 7
    elif((s >'08:00') & (s<='09:00')):
        return 8
    elif((s >'09:00') & (s<='10:00')):
        return 9
    elif((s >'10:00') & (s<='11:00')):
        return 10
    elif((s >'11:00') & (s<='12:00')):
        return 11
    elif((s >'12:00') & (s<='13:00')):
        return 12
    elif((s >'13:00') & (s<='14:00')):
        return 13
    elif((s >'14:00') & (s<='15:00')):
        return 14
    elif((s >'15:00') & (s<='16:00')):
        return 15
    elif((s >'16:00') & (s<='17:00')):
        return 16
    elif((s >'17:00') & (s<='18:00')):
        return 17
    elif((s >'18:00') & (s<='19:00')):
        return 18
    elif((s >'19:00') & (s<='20:00')):
        return 19
    elif((s >'20:00') & (s<='21:00')):
        return 20
    elif((s >'21:00') & (s<='22:00')):
        return 21
    elif((s >'22:00') & (s<='23:00')):
        return 22
    elif((s >'23:00') & (s<='24:00')):
        return 23

train_path = path + 'train/'
test_path = path + 'valid/'


train_status_lines = open(train_path + 'train_status.txt',encoding='UTF-8').readlines()
train_dict = dict()
early_flag = 0
for train_status in train_status_lines:
    r = train_status.split(',')
    s = r[4].split()
    
    if(len(s)==1):
        early_flag = 1    
    else:    
        t = timePerHour(s[1])
        if((r[0] in train_dict) == False):
            train_dict[r[0]] = defaultdict(int)
        train_dict[r[0]][t] = train_dict[r[0]][t] + 1
        if(early_flag==1):
            train_dict[r[0]][t] = train_dict[r[0]][t] + 1
            early_flag = 0
train_dict_pd = pd.DataFrame(train_dict).T.reset_index().rename(columns={"index": "uid"})
train_dict_pd = train_dict_pd.drop(None, axis=1)
train_dict_pd.columns = 'timePeriod_1hour_' + train_dict_pd.columns.astype(str)
train_dict_pd = train_dict_pd.rename(columns={"timePeriod_1hour_uid": "uid"})
train_dict_pd = train_dict_pd.fillna(0)

test_status_lines = open(test_path + 'valid_status.txt',encoding='UTF-8').readlines()
test_dict = dict()
early_flag = 0
for test_status in test_status_lines:
    r = test_status.split(',')
    s = r[4].split()
    
    if(len(s)==1):
        early_flag = 1    
    else:    
        t = timePerHour(s[1])
        if((r[0] in test_dict) == False):
            test_dict[r[0]] = defaultdict(int)
        test_dict[r[0]][t] = test_dict[r[0]][t] + 1
        if(early_flag==1):
            test_dict[r[0]][t] = test_dict[r[0]][t] + 1
            early_flag = 0
test_dict_pd = pd.DataFrame(test_dict).T.reset_index().rename(columns={"index": "uid"})
test_dict_pd = test_dict_pd.drop(None, axis=1)
test_dict_pd.columns = 'timePeriod_1hour_' + test_dict_pd.columns.astype(str)
test_dict_pd = test_dict_pd.rename(columns={"timePeriod_1hour_uid": "uid"})
test_dict_pd = test_dict_pd.fillna(0)

feat_time_1hour = pd.concat([train_dict_pd, test_dict_pd], axis = 0, ignore_index = True )


# ---save time feature---
feat_time_3hour.to_csv(path+'newfeat/feat_time_3hour.csv', header='infer', sep = ',', index = 0)
feat_time_1hour.to_csv(path+'newfeat/feat_time_1hour.csv', header='infer', sep = ',', index = 0)
